In [1]:
from __future__ import print_function

import argparse
import pickle

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                                ])
trainset_labeled_import = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
trainset_unlabeled_import = pickle.load(open("../data/kaggle/train_unlabeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))
train_labeled_loader = torch.utils.data.DataLoader(trainset_labeled_import, batch_size=32, shuffle=True)
train_unlabeled_loader = torch.utils.data.DataLoader(trainset_unlabeled_import, batch_size=256,
                                                     shuffle=True)

train_unlabeled_loader.dataset.train_labels = [-1 for i in range(len(train_unlabeled_loader.dataset.train_data))]
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=32, shuffle=True)

Files already downloaded
3000
750


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.sigmoid(x)
    

In [4]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=.01, momentum=.8)


In [43]:

class MyCriterion(torch.autograd.Function):
    def __init__(self):
        self.alpha = .0005
    
    def forward(self, input, target, epoch, isLabeled):
        loss = F.cross_entropy(input, target)

        self.save_for_backward(input, target, epoch, isLabeled, loss)
        if (isLabeled.data > 0).all():

            return Variable(loss.data * self.alpha * epoch.data)
        return loss

    def backward(self, grad_output):
       
        input, target, epoch, isLabeled, loss, = self.saved_tensors
        grad_input = loss.backward() 
        return grad_input
my_criterion = MyCriterion()
x = Variable(torch.randn(11, 10).type(torch.FloatTensor))
y = Variable(torch.range(1,6, .5).type(torch.LongTensor))

a = torch.from_numpy(np.array([0]))
b = torch.from_numpy(np.array([1]))
c = torch.from_numpy(np.array([10.0]))

# print(torch.from_numpy(np.array([10])))
print(my_criterion.forward(x, y, Variable(c.float()),  Variable(a)))
print(my_criterion.forward(x, y, Variable(c.float()),  Variable(b)))



Variable containing:
 2.7365
[torch.FloatTensor of size 1]

Variable containing:
1.00000e-02 *
  1.3683
[torch.FloatTensor of size 1]



In [10]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_labeled_loader):

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)

        loss = my_criterion.forward(output, target, Variable(torch.LongTensor(epoch)), Variable(torch.LongTensor(1)))
        my_criterion.backward(loss)
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Labeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_labeled_loader.dataset),
                       100. * batch_idx / len(train_labeled_loader), loss.data[0]))

    for batch_idx, (data,_) in enumerate(train_unlabeled_loader):
        data = Variable(data)
        optimizer.zero_grad()
        output = model(data)
        target = Variable(torch.LongTensor(output.data.max(1)[1].numpy().reshape(-1)))
        loss = my_criterion.forward(output, target, Variable(torch.LongTensor(epoch)), Variable(torch.LongTensor(2)))
        my_criterion.backward(loss)
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Unlabeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_unlabeled_loader.dataset),
                       100. * batch_idx / len(train_unlabeled_loader), loss.data[0]))

In [11]:
def validate(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += my_criterion.forward(output, target, Variable(epoch), Variable(torch.LongTensor(1))).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [12]:
for epoch in range(1, 2):
    train(epoch)
    validate(epoch, valid_loader)

RuntimeError: Variable data has to be a tensor, but got Variable

In [ ]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [15]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    
    label_predict = np.concatenate((label_predict, temp))

In [53]:
label_predict

predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

predict_label.head()

predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)